In [1]:
%matplotlib inline

import numpy as np
import time
import os

import torch
import torch.nn as nn
from torch.nn.modules.module import Module
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import argparse
import torch.nn.functional as F
from torch.utils.data.dataloader import _use_shared_memory
from torch.nn import Parameter
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import scipy
from scipy.sparse import coo_matrix
import pdb

from tensorboardX import SummaryWriter
import os
import csv

import matplotlib.pyplot as plt


import matplotlib
import matplotlib.colors as colors
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import networkx as nx
import sklearn.metrics as metrics
import torch
import torch.nn as nn
from torch.autograd import Variable
from tensorboardX import SummaryWriter

import argparse
import os
import pickle
import random
import shutil
import time

import cross_val
import encoders
import gen.feat as featgen
import gen.data as datagen
from graph_sampler import GraphSampler
import load_data
import util

In [2]:
class Csv_Logger():
    def __init__(self, log_path, log_header):
        self.log_path = os.path.abspath(log_path)
        self.log_header = log_header

        self.initialized = False

    def _initialize(self):
        self.initialized = True
        parent_dir = os.path.abspath(os.path.join(self.log_path, '..'))
        os.makedirs(parent_dir, exist_ok=True)

        with open(self.log_path, "w") as f:
            writer = csv.DictWriter(f, self.log_header)
            writer.writeheader()

    def write_row(self, **kwargs):
        if not self.initialized:
            self._initialize()

        with open(self.log_path, "a") as f:
            writer = csv.DictWriter(f, self.log_header)
            writer.writerow(kwargs)


In [3]:
def load_data():
    dayX=np.load('../preprocessed_data/daysX.npy').reshape(-1,24,9413).astype(int) #(2738, 9413, 24)
    ydaysD=np.load('../preprocessed_data/ydaysD.npy').astype(int) #(2738, 1)
    x_train, x_test, y_train, y_test = train_test_split(dayX, ydaysD, test_size=0.2)
    y_test=y_test.squeeze()
    y_train=y_train.squeeze()
    adj_load=scipy.io.loadmat('AdjG.mat')['AdjG']    
    adj=adj_load.astype(float)
    return adj,x_train, x_test, y_train, y_test

In [4]:
class Dataset(Dataset):
    # Custom dataset
    def __init__(self, features, labels):
        labels=labels[:,np.newaxis]
        self.features = [torch.from_numpy(x) for x in features]
        self.labels = [torch.from_numpy(x) for x in labels]
        assert len(self.features) == len(self.labels)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, item): #for-loop yield 
        return self.features[item], self.labels[item]

In [ ]:
def get_adjacency(order_num,adj,device):
        #Identity
    ret =[]
    c=scipy.sparse.identity(9413)
    for i in range(order_num+1): #indices, values
        temp=coo_matrix(c)
        values=temp.data
        indices = np.vstack((temp.row, temp.col))
        i = torch.LongTensor(indices).to(device)
        v = torch.FloatTensor(values).to(device)
        adj_v=torch.sparse.FloatTensor(i, v, torch.Size(temp.shape))
        adj_v.require_grad=False
        adj_v.to(device)
        ret.append(adj_v)
        c=c*adj
    return ret

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [ ]:
class TAGCN_layer(Module):

    def __init__(self, in_channels, out_channels,degree,adj_matrices, bias=True,include_identity=False):
        super(TAGCN_layer, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.weight = Parameter(torch.FloatTensor(degree+1,in_channels,out_channels))
        self.degree=degree
        self.bias = Parameter(torch.FloatTensor(out_channels))#parameter and variable
        self.reset_parameters_xavier()
        self.adj_matrices=adj_matrices
        self.include_identity=include_identity

        
    def reset_parameters_xavier(self):
        nn.init.xavier_normal_(self.weight.data,gain=0.02) # Implement Xavier Uniform, 0.02
#         nn.init.xavier_uniform_(self.weight.data,gain=0.02) # Implement Xavier Uniform, 0.02

        nn.init.constant_(self.bias.data, 0.0)


    def forward(self, inputs):
        # x = inputs # samples by channels by 9413
        [number_of_samples,in_channels, number_of_nodes]=inputs.shape
        assert (in_channels==self.in_channels)
        output=torch.zeros(number_of_nodes,number_of_samples,self.out_channels).to(device)

        for out_channel in range(self.out_channels):
            for i in range(self.degree):
                test=torch.sum(self.weight[i+1,:,out_channel]*inputs.transpose(1,2),dim=2).view(number_of_nodes,-1)
                output[:,:,out_channel] += torch.spmm(self.adj_matrices[i+1],test).to(device)

            output[:,:,out_channel]=output[:,:,out_channel]+self.bias[out_channel]
        return output.view(number_of_samples,self.out_channels,number_of_nodes).squeeze()


In [ ]:
class Trainer:
    def __init__(self, model, optimizer,loader, max_epochs=100,load_path=None):
        self.model = model
        self.loader = loader
        self.train_losses = []
        self.val_losses = []
        self.predictions = []
        self.predictions_test = []
        self.generated_logits = []
        self.generated = []
        self.generated_logits_test = []
        self.generated_test = []
        self.epochs = 0
        self.max_epochs = max_epochs
        self.optimizer = optimizer
        self.criterion = nn.CrossEntropyLoss()
        self.testaccuracies=[]
        self.trainaccuracies=[]
        self.name='./logs/'+str(time.time())+'.csv'
        self.logger = Csv_Logger(self.name, ['episode', 'test_accuracy', 'train_accuracy', 'train_loss'])


        if load_path is not None:
            self.model.load_state_dict(
                torch.load(
                    load_path,
                    map_location=lambda storage,
                    loc: storage))
            
    def train(self,device):
        self.model.train() # set to training mode
        epoch_loss = 0
        correct = 0
        for batch_num, (features,labels) in enumerate(self.loader):
            self.optimizer.zero_grad()
            bs=features.shape[0]
            features=features.to(device).float()
            labels=labels.to(device).long().squeeze()
            self.model=self.model.to(device)
            
            out=self.model.forward(features)  
            out=out.view(bs,7)
            pred = out.data.max(1, keepdim=True)[1]
            predicted = pred.eq(labels.data.view_as(pred))
            correct += predicted.sum()            
            loss = self.criterion(out,labels)
            loss.backward()
            self.optimizer.step()
            epoch_loss += loss.item()

        epoch_loss = epoch_loss / (batch_num + 1)
        writer.add_scalar('loss',epoch_loss,self.epochs)

        accuracy_train = correct.cpu().numpy() / len(self.loader.dataset)
        writer.add_scalar('train accuracy',accuracy_train,self.epochs)

        self.epochs += 1
        print('[TRAIN]  Epoch [%d/%d]   Loss: %.4f     Accuracy: %.4f'
                      % (self.epochs, self.max_epochs, epoch_loss,accuracy_train))

        self.train_losses.append(epoch_loss)
        self.trainaccuracies.append(accuracy_train)
    def save_model(self, path):
        torch.save(self.model.state_dict(), path)
        
    def inference(self,inference_loader,device):
        self.model = self.model.eval()
        bs=64
        y_true=[]
        y_pred=[]

        with torch.no_grad():
            correct=0
            for batch_num, (features,labels) in enumerate(inference_loader):
#                 self.optimizer.zero_grad()
                bs=features.shape[0]
                features=features.to(device).float()
                labels=labels.to(device).long().squeeze()
                self.model=self.model.to(device)
                out=self.model.forward(features)  
                out=out.view(bs,7)
                pred = out.data.max(1, keepdim=True)[1]
                predicted = pred.eq(labels.data.view_as(pred))
                correct += predicted.sum()
                y_true=y_true+labels.cpu().numpy().tolist()
                y_pred=y_pred+pred.cpu().numpy().reshape(-1).tolist()
            conf_matrix=confusion_matrix(y_true,y_pred)
        
            accuracy_test = correct.cpu().numpy() / len(inference_loader.dataset)
            self.testaccuracies.append(accuracy_test)
            print('[Test]  Epoch [%d/%d]   Accuracy: %.4f'
                      % (self.epochs, self.max_epochs,accuracy_test))

        self.logger.write_row(episode=self.epochs, test_accuracy=accuracy_test,train_accuracy=self.trainaccuracies[-1],train_loss=self.train_losses[-1])

#             print('Confusion Matrix: ',conf_matrix)

In [3]:
def evaluate(dataset, model, args, name='Validation', max_num_examples=None):
    model.eval()

    labels = []
    preds = []
    for batch_idx, data in enumerate(dataset):
        adj = Variable(data['adj'].float(), requires_grad=False).cuda()
        h0 = Variable(data['feats'].float()).cuda()
        labels.append(data['label'].long().numpy())
        batch_num_nodes = data['num_nodes'].int().numpy()
        assign_input = Variable(data['assign_feats'].float(), requires_grad=False).cuda()

        ypred = model(h0, adj, batch_num_nodes, assign_x=assign_input)
        _, indices = torch.max(ypred, 1)
        preds.append(indices.cpu().data.numpy())

        if max_num_examples is not None:
            if (batch_idx+1)*args.batch_size > max_num_examples:
                break

    labels = np.hstack(labels)
    preds = np.hstack(preds)
    
    result = {'prec': metrics.precision_score(labels, preds, average='macro'),
              'recall': metrics.recall_score(labels, preds, average='macro'),
              'acc': metrics.accuracy_score(labels, preds),
              'F1': metrics.f1_score(labels, preds, average="micro")}
    print(name, " accuracy:", result['acc'])
    return result

def gen_prefix(args):
    if args.bmname is not None:
        name = args.bmname
    else:
        name = args.dataset
    name += '_' + args.method
    if args.method == 'soft-assign':
        name += '_l' + str(args.num_gc_layers) + 'x' + str(args.num_pool)
        name += '_ar' + str(int(args.assign_ratio*100))
        if args.linkpred:
            name += '_lp'
    else:
        name += '_l' + str(args.num_gc_layers)
    name += '_h' + str(args.hidden_dim) + '_o' + str(args.output_dim)
    if not args.bias:
        name += '_nobias'
    if len(args.name_suffix) > 0:
        name += '_' + args.name_suffix
    return name

def gen_train_plt_name(args):
    return 'results/' + gen_prefix(args) + '.png'

def log_assignment(assign_tensor, writer, epoch, batch_idx):
    plt.switch_backend('agg')
    fig = plt.figure(figsize=(8,6), dpi=300)

    # has to be smaller than args.batch_size
    for i in range(len(batch_idx)):
        plt.subplot(2, 2, i+1)
        plt.imshow(assign_tensor.cpu().data.numpy()[batch_idx[i]], cmap=plt.get_cmap('BuPu'))
        cbar = plt.colorbar()
        cbar.solids.set_edgecolor("face")
    plt.tight_layout()
    fig.canvas.draw()

    data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    writer.add_image('assignment', data, epoch)

def log_graph(adj, batch_num_nodes, writer, epoch, batch_idx, assign_tensor=None):
    plt.switch_backend('agg')
    fig = plt.figure(figsize=(8,6), dpi=300)

    for i in range(len(batch_idx)):
        ax = plt.subplot(2, 2, i+1)
        num_nodes = batch_num_nodes[batch_idx[i]]
        adj_matrix = adj[batch_idx[i], :num_nodes, :num_nodes].cpu().data.numpy()
        G = nx.from_numpy_matrix(adj_matrix)
        nx.draw(G, pos=nx.spring_layout(G), with_labels=True, node_color='#336699',
                edge_color='grey', width=0.5, node_size=300,
                alpha=0.7)
        ax.xaxis.set_visible(False)

    plt.tight_layout()
    fig.canvas.draw()

    data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    writer.add_image('graphs', data, epoch)

    # log a label-less version
    #fig = plt.figure(figsize=(8,6), dpi=300)
    #for i in range(len(batch_idx)):
    #    ax = plt.subplot(2, 2, i+1)
    #    num_nodes = batch_num_nodes[batch_idx[i]]
    #    adj_matrix = adj[batch_idx[i], :num_nodes, :num_nodes].cpu().data.numpy()
    #    G = nx.from_numpy_matrix(adj_matrix)
    #    nx.draw(G, pos=nx.spring_layout(G), with_labels=False, node_color='#336699',
    #            edge_color='grey', width=0.5, node_size=25,
    #            alpha=0.8)

    #plt.tight_layout()
    #fig.canvas.draw()

    #data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    #data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    #writer.add_image('graphs_no_label', data, epoch)

    # colored according to assignment
    assignment = assign_tensor.cpu().data.numpy()
    fig = plt.figure(figsize=(8,6), dpi=300)

    num_clusters = assignment.shape[2]
    all_colors = np.array(range(num_clusters))

    for i in range(len(batch_idx)):
        ax = plt.subplot(2, 2, i+1)
        num_nodes = batch_num_nodes[batch_idx[i]]
        adj_matrix = adj[batch_idx[i], :num_nodes, :num_nodes].cpu().data.numpy()

        label = np.argmax(assignment[batch_idx[i]], axis=1).astype(int)
        label = label[: batch_num_nodes[batch_idx[i]]]
        node_colors = all_colors[label]

        G = nx.from_numpy_matrix(adj_matrix)
        nx.draw(G, pos=nx.spring_layout(G), with_labels=False, node_color=node_colors,
                edge_color='grey', width=0.4, node_size=50, cmap=plt.get_cmap('Set1'),
                vmin=0, vmax=num_clusters-1,
                alpha=0.8)

    plt.tight_layout()
    fig.canvas.draw()

    data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    writer.add_image('graphs_colored', data, epoch)


def train(dataset, model, args, same_feat=True, val_dataset=None, test_dataset=None, writer=None,
        mask_nodes = True):
    writer_batch_idx = [0, 3, 6, 9]
    
    optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, model.parameters()), lr=0.001)
    iter = 0
    best_val_result = {
            'epoch': 0,
            'loss': 0,
            'acc': 0}
    test_result = {
            'epoch': 0,
            'loss': 0,
            'acc': 0}
    train_accs = []
    train_epochs = []
    best_val_accs = []
    best_val_epochs = []
    test_accs = []
    test_epochs = []
    val_accs = []
    for epoch in range(args.num_epochs):
        begin_time = time.time()
        avg_loss = 0.0
        model.train()
        print('Epoch: ', epoch)
        for batch_idx, data in enumerate(dataset):
            model.zero_grad()
            adj = Variable(data['adj'].float(), requires_grad=False).cuda()
            h0 = Variable(data['feats'].float(), requires_grad=False).cuda()
            label = Variable(data['label'].long()).cuda()
            batch_num_nodes = data['num_nodes'].int().numpy() if mask_nodes else None
            assign_input = Variable(data['assign_feats'].float(), requires_grad=False).cuda()

            ypred = model(h0, adj, batch_num_nodes, assign_x=assign_input)
            if not args.method == 'soft-assign' or not args.linkpred:
                loss = model.loss(ypred, label)
            else:
                loss = model.loss(ypred, label, adj, batch_num_nodes)
            loss.backward()
            nn.utils.clip_grad_norm(model.parameters(), args.clip)
            optimizer.step()
            iter += 1
            avg_loss += loss
            #if iter % 20 == 0:
            #    print('Iter: ', iter, ', loss: ', loss.data[0])

            # log once per XX epochs
            if epoch % 10 == 0 and batch_idx == len(dataset) // 2 and args.method == 'soft-assign' and writer is not None:
                log_assignment(model.assign_tensor, writer, epoch, writer_batch_idx)
                log_graph(adj, batch_num_nodes, writer, epoch, writer_batch_idx, model.assign_tensor)
        avg_loss /= batch_idx + 1
        elapsed = time.time() - begin_time
        if writer is not None:
            writer.add_scalar('loss/avg_loss', avg_loss, epoch)
            if args.linkpred:
                writer.add_scalar('loss/linkpred_loss', model.link_loss, epoch)
        print('Avg loss: ', avg_loss, '; epoch time: ', elapsed)
        result = evaluate(dataset, model, args, name='Train', max_num_examples=100)
        train_accs.append(result['acc'])
        train_epochs.append(epoch)
        if val_dataset is not None:
            val_result = evaluate(val_dataset, model, args, name='Validation')
            val_accs.append(val_result['acc'])
        if val_result['acc'] > best_val_result['acc'] - 1e-7:
            best_val_result['acc'] = val_result['acc']
            best_val_result['epoch'] = epoch
            best_val_result['loss'] = avg_loss
        if test_dataset is not None:
            test_result = evaluate(test_dataset, model, args, name='Test')
            test_result['epoch'] = epoch
        if writer is not None:
            writer.add_scalar('acc/train_acc', result['acc'], epoch)
            writer.add_scalar('acc/val_acc', val_result['acc'], epoch)
            writer.add_scalar('loss/best_val_loss', best_val_result['loss'], epoch)
            if test_dataset is not None:
                writer.add_scalar('acc/test_acc', test_result['acc'], epoch)

        print('Best val result: ', best_val_result)
        best_val_epochs.append(best_val_result['epoch'])
        best_val_accs.append(best_val_result['acc'])
        if test_dataset is not None:
            print('Test result: ', test_result)
            test_epochs.append(test_result['epoch'])
            test_accs.append(test_result['acc'])

    matplotlib.style.use('seaborn')
    plt.switch_backend('agg')
    plt.figure()
    plt.plot(train_epochs, util.exp_moving_avg(train_accs, 0.85), '-', lw=1)
    if test_dataset is not None:
        plt.plot(best_val_epochs, best_val_accs, 'bo', test_epochs, test_accs, 'go')
        plt.legend(['train', 'val', 'test'])
    else:
        plt.plot(best_val_epochs, best_val_accs, 'bo')
        plt.legend(['train', 'val'])
    plt.savefig(gen_train_plt_name(args), dpi=600)
    plt.close()
    matplotlib.style.use('default')

    return model, val_accs

def prepare_data(graphs, args, test_graphs=None, max_nodes=0):

    random.shuffle(graphs)
    if test_graphs is None:
        train_idx = int(len(graphs) * args.train_ratio)
        test_idx = int(len(graphs) * (1-args.test_ratio))
        train_graphs = graphs[:train_idx]
        val_graphs = graphs[train_idx: test_idx]
        test_graphs = graphs[test_idx:]
    else:
        train_idx = int(len(graphs) * args.train_ratio)
        train_graphs = graphs[:train_idx]
        val_graphs = graph[train_idx:]
    print('Num training graphs: ', len(train_graphs), 
          '; Num validation graphs: ', len(val_graphs),
          '; Num testing graphs: ', len(test_graphs))

    print('Number of graphs: ', len(graphs))
    print('Number of edges: ', sum([G.number_of_edges() for G in graphs]))
    print('Max, avg, std of graph size: ', 
            max([G.number_of_nodes() for G in graphs]), ', '
            "{0:.2f}".format(np.mean([G.number_of_nodes() for G in graphs])), ', '
            "{0:.2f}".format(np.std([G.number_of_nodes() for G in graphs])))

    # minibatch
    dataset_sampler = GraphSampler(train_graphs, normalize=False, max_num_nodes=max_nodes,
            features=args.feature_type)
    train_dataset_loader = torch.utils.data.DataLoader(
            dataset_sampler, 
            batch_size=args.batch_size, 
            shuffle=True,
            num_workers=args.num_workers)

    dataset_sampler = GraphSampler(val_graphs, normalize=False, max_num_nodes=max_nodes,
            features=args.feature_type)
    val_dataset_loader = torch.utils.data.DataLoader(
            dataset_sampler, 
            batch_size=args.batch_size, 
            shuffle=False,
            num_workers=args.num_workers)

    dataset_sampler = GraphSampler(test_graphs, normalize=False, max_num_nodes=max_nodes,
            features=args.feature_type)
    test_dataset_loader = torch.utils.data.DataLoader(
            dataset_sampler, 
            batch_size=args.batch_size, 
            shuffle=False,
            num_workers=args.num_workers)

    return train_dataset_loader, val_dataset_loader, test_dataset_loader, \
            dataset_sampler.max_num_nodes, dataset_sampler.feat_dim, dataset_sampler.assign_feat_dim

def syn_community1v2(args, writer=None, export_graphs=False):

    # data
    graphs1 = datagen.gen_ba(range(40, 60), range(4, 5), 500, 
            featgen.ConstFeatureGen(np.ones(args.input_dim, dtype=float)))
    for G in graphs1:
        G.graph['label'] = 0
    if export_graphs:
        util.draw_graph_list(graphs1[:16], 4, 4, 'figs/ba')

    graphs2 = datagen.gen_2community_ba(range(20, 30), range(4, 5), 500, 0.3, 
            [featgen.ConstFeatureGen(np.ones(args.input_dim, dtype=float))])
    for G in graphs2:
        G.graph['label'] = 1
    if export_graphs:
        util.draw_graph_list(graphs2[:16], 4, 4, 'figs/ba2')

    graphs = graphs1 + graphs2
    
    train_dataset, val_dataset, test_dataset, max_num_nodes, input_dim, assign_input_dim = prepare_data(graphs, args)
    if args.method == 'soft-assign':
        print('Method: soft-assign')
        model = encoders.SoftPoolingGcnEncoder(
                max_num_nodes, 
                input_dim, args.hidden_dim, args.output_dim, args.num_classes, args.num_gc_layers,
                args.hidden_dim, assign_ratio=args.assign_ratio, num_pooling=args.num_pool,
                bn=args.bn, linkpred=args.linkpred, assign_input_dim=assign_input_dim).cuda()
    elif args.method == 'base-set2set':
        print('Method: base-set2set')
        model = encoders.GcnSet2SetEncoder(input_dim, args.hidden_dim, args.output_dim, 2,
                args.num_gc_layers, bn=args.bn).cuda()
    else:
        print('Method: base')
        model = encoders.GcnEncoderGraph(input_dim, args.hidden_dim, args.output_dim, 2,
                args.num_gc_layers, bn=args.bn).cuda()

    train(train_dataset, model, args, val_dataset=val_dataset, test_dataset=test_dataset,
            writer=writer)

def syn_community2hier(args, writer=None):

    # data
    feat_gen = [featgen.ConstFeatureGen(np.ones(args.input_dim, dtype=float))]
    graphs1 = datagen.gen_2hier(1000, [2,4], 10, range(4,5), 0.1, 0.03, feat_gen)
    graphs2 = datagen.gen_2hier(1000, [3,3], 10, range(4,5), 0.1, 0.03, feat_gen)
    graphs3 = datagen.gen_2community_ba(range(28, 33), range(4,7), 1000, 0.25, feat_gen)

    for G in graphs1:
        G.graph['label'] = 0
    for G in graphs2:
        G.graph['label'] = 1
    for G in graphs3:
        G.graph['label'] = 2

    graphs = graphs1 + graphs2 + graphs3

    train_dataset, val_dataset, test_dataset, max_num_nodes, input_dim, assign_input_dim = prepare_data(graphs, args)

    if args.method == 'soft-assign':
        print('Method: soft-assign')
        model = encoders.SoftPoolingGcnEncoder(
                max_num_nodes, 
                input_dim, args.hidden_dim, args.output_dim, args.num_classes, args.num_gc_layers,
                args.hidden_dim, assign_ratio=args.assign_ratio, num_pooling=args.num_pool,
                bn=args.bn, linkpred=args.linkpred, args=args, assign_input_dim=assign_input_dim).cuda()
    elif args.method == 'base-set2set':
        print('Method: base-set2set')
        model = encoders.GcnSet2SetEncoder(input_dim, args.hidden_dim, args.output_dim, 2,
                args.num_gc_layers, bn=args.bn, args=args, assign_input_dim=assign_input_dim).cuda()
    else:
        print('Method: base')
        model = encoders.GcnEncoderGraph(input_dim, args.hidden_dim, args.output_dim, 2,
                args.num_gc_layers, bn=args.bn, args=args).cuda()
    train(train_dataset, model, args, val_dataset=val_dataset, test_dataset=test_dataset,
            writer=writer)


def pkl_task(args, feat=None):
    with open(os.path.join(args.datadir, args.pkl_fname), 'rb') as pkl_file:
        data = pickle.load(pkl_file)
    graphs = data[0]
    labels = data[1]
    test_graphs = data[2]
    test_labels = data[3]

    for i in range(len(graphs)):
        graphs[i].graph['label'] = labels[i]
    for i in range(len(test_graphs)):
        test_graphs[i].graph['label'] = test_labels[i]

    if feat is None:
        featgen_const = featgen.ConstFeatureGen(np.ones(args.input_dim, dtype=float))
        for G in graphs:
            featgen_const.gen_node_features(G)
        for G in test_graphs:
            featgen_const.gen_node_features(G)

    train_dataset, test_dataset, max_num_nodes = prepare_data(graphs, args, test_graphs=test_graphs)
    model = encoders.GcnEncoderGraph(
            args.input_dim, args.hidden_dim, args.output_dim, args.num_classes, 
            args.num_gc_layers, bn=args.bn).cuda()
    train(train_dataset, model, args, test_dataset=test_dataset)
    evaluate(test_dataset, model, args, 'Validation')

def benchmark_task(args, writer=None, feat='node-label'):
    graphs = load_data.read_graphfile(args.datadir, args.bmname, max_nodes=args.max_nodes)
    
    if feat == 'node-feat' and 'feat_dim' in graphs[0].graph:
        print('Using node features')
        input_dim = graphs[0].graph['feat_dim']
    elif feat == 'node-label' and 'label' in graphs[0].node[0]:
        print('Using node labels')
        for G in graphs:
            for u in G.nodes():
                G.node[u]['feat'] = np.array(G.node[u]['label'])
    else:
        print('Using constant labels')
        featgen_const = featgen.ConstFeatureGen(np.ones(args.input_dim, dtype=float))
        for G in graphs:
            featgen_const.gen_node_features(G)

    train_dataset, val_dataset, test_dataset, max_num_nodes, input_dim, assign_input_dim = \
            prepare_data(graphs, args, max_nodes=args.max_nodes)
    if args.method == 'soft-assign':
        print('Method: soft-assign')
        model = encoders.SoftPoolingGcnEncoder(
                max_num_nodes, 
                input_dim, args.hidden_dim, args.output_dim, args.num_classes, args.num_gc_layers,
                args.hidden_dim, assign_ratio=args.assign_ratio, num_pooling=args.num_pool,
                bn=args.bn, dropout=args.dropout, linkpred=args.linkpred, args=args,
                assign_input_dim=assign_input_dim).cuda()
    elif args.method == 'base-set2set':
        print('Method: base-set2set')
        model = encoders.GcnSet2SetEncoder(
                input_dim, args.hidden_dim, args.output_dim, args.num_classes,
                args.num_gc_layers, bn=args.bn, dropout=args.dropout, args=args).cuda()
    else:
        print('Method: base')
        model = encoders.GcnEncoderGraph(
                input_dim, args.hidden_dim, args.output_dim, args.num_classes, 
                args.num_gc_layers, bn=args.bn, dropout=args.dropout, args=args).cuda()

    train(train_dataset, model, args, val_dataset=val_dataset, test_dataset=test_dataset,
            writer=writer)
    evaluate(test_dataset, model, args, 'Validation')


def benchmark_task_val(args, writer=None, feat='node-label'):
    all_vals = []
    graphs = load_data.read_graphfile(args.datadir, args.bmname, max_nodes=args.max_nodes)
    
    if feat == 'node-feat' and 'feat_dim' in graphs[0].graph:
        print('Using node features')
        input_dim = graphs[0].graph['feat_dim']
    elif feat == 'node-label' and 'label' in graphs[0].node[0]:
        print('Using node labels')
        for G in graphs:
            for u in G.nodes():
                G.node[u]['feat'] = np.array(G.node[u]['label'])
    else:
        print('Using constant labels')
        featgen_const = featgen.ConstFeatureGen(np.ones(args.input_dim, dtype=float))
        for G in graphs:
            featgen_const.gen_node_features(G)

    for i in range(10):
        train_dataset, val_dataset, max_num_nodes, input_dim, assign_input_dim = \
                cross_val.prepare_val_data(graphs, args, i, max_nodes=args.max_nodes)
        if args.method == 'soft-assign':
            print('Method: soft-assign')
            model = encoders.SoftPoolingGcnEncoder(
                    max_num_nodes, 
                    input_dim, args.hidden_dim, args.output_dim, args.num_classes, args.num_gc_layers,
                    args.hidden_dim, assign_ratio=args.assign_ratio, num_pooling=args.num_pool,
                    bn=args.bn, dropout=args.dropout, linkpred=args.linkpred, args=args,
                    assign_input_dim=assign_input_dim).cuda()
        elif args.method == 'base-set2set':
            print('Method: base-set2set')
            model = encoders.GcnSet2SetEncoder(
                    input_dim, args.hidden_dim, args.output_dim, args.num_classes,
                    args.num_gc_layers, bn=args.bn, dropout=args.dropout, args=args).cuda()
        else:
            print('Method: base')
            model = encoders.GcnEncoderGraph(
                    input_dim, args.hidden_dim, args.output_dim, args.num_classes, 
                    args.num_gc_layers, bn=args.bn, dropout=args.dropout, args=args).cuda()

        _, val_accs = train(train_dataset, model, args, val_dataset=val_dataset, test_dataset=None,
            writer=writer)
        all_vals.append(np.array(val_accs))
    all_vals = np.vstack(all_vals)
    all_vals = np.mean(all_vals, axis=0)
    print(all_vals)
    print(np.max(all_vals))
    print(np.argmax(all_vals))
    
    
def arg_parse():
    parser = argparse.ArgumentParser(description='GraphPool arguments.')
    io_parser = parser.add_mutually_exclusive_group(required=False)
    io_parser.add_argument('--dataset', dest='dataset', 
            help='Input dataset.')
    benchmark_parser = io_parser.add_argument_group()
    benchmark_parser.add_argument('--bmname', dest='bmname',
            help='Name of the benchmark dataset')
    io_parser.add_argument('--pkl', dest='pkl_fname',
            help='Name of the pkl data file')

    softpool_parser = parser.add_argument_group()
    softpool_parser.add_argument('--assign-ratio', dest='assign_ratio', type=float,
            help='ratio of number of nodes in consecutive layers')
    softpool_parser.add_argument('--num-pool', dest='num_pool', type=int,
            help='number of pooling layers')
    parser.add_argument('--linkpred', dest='linkpred', action='store_const',
            const=True, default=False,
            help='Whether link prediction side objective is used')


    parser.add_argument('--datadir', dest='datadir',
            help='Directory where benchmark is located')
    parser.add_argument('--logdir', dest='logdir',
            help='Tensorboard log directory')
    parser.add_argument('--cuda', dest='cuda',
            help='CUDA.')
    parser.add_argument('--max-nodes', dest='max_nodes', type=int,
            help='Maximum number of nodes (ignore graghs with nodes exceeding the number.')
    parser.add_argument('--lr', dest='lr', type=float,
            help='Learning rate.')
    parser.add_argument('--clip', dest='clip', type=float,
            help='Gradient clipping.')
    parser.add_argument('--batch-size', dest='batch_size', type=int,
            help='Batch size.')
    parser.add_argument('--epochs', dest='num_epochs', type=int,
            help='Number of epochs to train.')
    parser.add_argument('--train-ratio', dest='train_ratio', type=float,
            help='Ratio of number of graphs training set to all graphs.')
    parser.add_argument('--num_workers', dest='num_workers', type=int,
            help='Number of workers to load data.')
    parser.add_argument('--feature', dest='feature_type',
            help='Feature used for encoder. Can be: id, deg')
    parser.add_argument('--input-dim', dest='input_dim', type=int,
            help='Input feature dimension')
    parser.add_argument('--hidden-dim', dest='hidden_dim', type=int,
            help='Hidden dimension')
    parser.add_argument('--output-dim', dest='output_dim', type=int,
            help='Output dimension')
    parser.add_argument('--num-classes', dest='num_classes', type=int,
            help='Number of label classes')
    parser.add_argument('--num-gc-layers', dest='num_gc_layers', type=int,
            help='Number of graph convolution layers before each pooling')
    parser.add_argument('--nobn', dest='bn', action='store_const',
            const=False, default=True,
            help='Whether batch normalization is used')
    parser.add_argument('--dropout', dest='dropout', type=float,
            help='Dropout rate.')
    parser.add_argument('--nobias', dest='bias', action='store_const',
            const=False, default=True,
            help='Whether to add bias. Default to True.')

    parser.add_argument('--method', dest='method',
            help='Method. Possible values: base, base-set2set, soft-assign')
    parser.add_argument('--name-suffix', dest='name_suffix',
            help='suffix added to the output filename')

    parser.set_defaults(datadir='data',
                        logdir='log',
                        dataset='syn1v2',
                        max_nodes=1000,
                        cuda='1',
                        feature_type='default',
                        lr=0.001,
                        clip=2.0,
                        batch_size=20,
                        num_epochs=1000,
                        train_ratio=0.8,
                        test_ratio=0.1,
                        num_workers=1,
                        input_dim=10,
                        hidden_dim=20,
                        output_dim=20,
                        num_classes=2,
                        num_gc_layers=3,
                        dropout=0.0,
                        method='base',
                        name_suffix='',
                        assign_ratio=0.1,
                        num_pool=1
                       )
    return parser.parse_args()

In [ ]:
# export scalar data to JSON for external processing
class prog_args_init:
    def __init__(self):
        self.datadir='data'
        self.logdir='log'
        self.dataset='syn1v2'
        self.max_nodes=1000
        self.cuda='1'
        self.feature_type='default'
        self.lr=0.001
        self.clip=2.0
        self.batch_size=20
        self.num_epochs=1000
        self.train_ratio=0.8
        self.test_ratio=0.1
        self.num_workers=0
        self.input_dim=10
        self.hidden_dim=20
        self.output_dim=20
        self.num_classes=2
        self.num_gc_layers=3
        self.dropout=0.0
        self.method='base'
        self.name_suffix=''
        self.assign_ratio=0.1
        self.num_pool=1
        self.bmname=None
        self.bias=None
        self.pkl_fname=None
        self.bn=None
        self.linkpred=None
prog_args=prog_args_init()     
path = os.path.join(prog_args.logdir, gen_prefix(prog_args))
if os.path.isdir(path):
    print('Remove existing log dir: ', path)
    shutil.rmtree(path)
writer = SummaryWriter(path)
#writer = None

os.environ['CUDA_VISIBLE_DEVICES'] = prog_args.cuda
print('CUDA', prog_args.cuda)

if prog_args.bmname is not None:
    benchmark_task_val(prog_args, writer=writer)
elif prog_args.pkl_fname is not None:
    pkl_task(prog_args)
elif prog_args.dataset is not None:
    if prog_args.dataset == 'syn1v2':
        syn_community1v2(prog_args, writer=writer)
    if prog_args.dataset == 'syn2hier':
        syn_community2hier(prog_args, writer=writer)

writer.close()

Remove existing log dir:  log/syn1v2_base_l3_h20_o20_nobias
CUDA 1
Num training graphs:  800 ; Num validation graphs:  100 ; Num testing graphs:  100
Number of graphs:  1000
Number of edges:  176060
Max, avg, std of graph size:  59 , 49.10 , 4.90
Method: base
Epoch:  0


/home/mark/Ph.D. Research/Projects/GCN/gcn_env/lib/python3.6/site-packages/ipykernel_launcher.py:181: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Avg loss:  tensor(0.6946, device='cuda:0', grad_fn=<DivBackward0>) ; epoch time:  0.7390921115875244
Train  accuracy: 0.49166666666666664
Validation  accuracy: 0.47
Test  accuracy: 0.42
Best val result:  {'epoch': 0, 'loss': tensor(0.6946, device='cuda:0', grad_fn=<DivBackward0>), 'acc': 0.47}
Test result:  {'prec': 0.21, 'recall': 0.5, 'acc': 0.42, 'F1': 0.41999999999999993, 'epoch': 0}
Epoch:  1


In [ ]:
# adj,x_train, x_test, y_train, y_test=load_data()
# number_of_nodes=9413
# number_of_classes=7
# writer = SummaryWriter()

In [ ]:
# #hyperparameters
# EPOCHS=400
# lr=1e-3
# # wd=1.2e-6
# # wd=1e-6
# # batch_size=2048
# batch_size=256
# include_identity=False 


In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# train_dataset=Dataset(x_train,y_train)
# train_loader = DataLoader(train_dataset,shuffle=True, batch_size=batch_size)
# test_dataset=Dataset(x_test,y_test)
# test_loader = DataLoader(test_dataset,shuffle=False, batch_size=batch_size)

In [ ]:
# epoch=0
# for degree in [3]: #,3,4 #1,2,3,4
# #     for dropout_rate in [0.2,0.5,0.8]:
#     print('Degree: ', degree)
#     adj_matrices=get_adjacency(degree,adj,device)
#     model=nn.Sequential(TAGCN_layer(24,1,degree,adj_matrices,include_identity),nn.ELU(),nn.Linear(number_of_nodes,number_of_classes)) 
# #     model=nn.Sequential(TAGCN_layer(24,1,degree,adj_matrices,include_identity),nn.ReLU(),nn.BatchNorm1d(number_of_nodes), nn.Linear(number_of_nodes,number_of_classes)) 
# #     model=nn.Sequential(TAGCN_layer(24,1,degree,adj_matrices,include_identity),nn.ReLU(),nn.Linear(number_of_nodes,number_of_classes)) 

    
# #     model=nn.Sequential(TAGCN_layer(24,24,degree,adj_matrices,include_identity),nn.ELU(),TAGCN_layer(24,8,degree,adj_matrices,include_identity),nn.ELU(),TAGCN_layer(8,1,degree,adj_matrices,include_identity),nn.ELU(),nn.Linear(number_of_nodes,number_of_classes)) 
# #     model=nn.Sequential(TAGCN_layer(24,1,degree,adj_matrices,include_identity),nn.ELU(),nn.Linear(number_of_nodes,number_of_classes)) 
#     print('Number of parameters: ', count_parameters(model))
#     AdamOptimizer = torch.optim.Adam(model.parameters())
#     trainer=Trainer(model,AdamOptimizer,train_loader, max_epochs=EPOCHS)
#     epoch=0
#     while epoch<EPOCHS:
#         ts=time.time()
#         epoch+=1
#         trainer.train(device)
#         trainer.inference(test_loader,device)
#         te=time.time()
#         print(te-ts)

# #     del adj_matrices

# #     print(np.mean(trainer.testaccuracies))
# #     print(np.var(trainer.testaccuracies))

Degree:  3
Number of parameters:  65995
[TRAIN]  Epoch [1/400]   Loss: 2.6816     Accuracy: 0.1479
[Test]  Epoch [1/400]   Accuracy: 0.2336
8.668795108795166
[TRAIN]  Epoch [2/400]   Loss: 2.2607     Accuracy: 0.2489
[Test]  Epoch [2/400]   Accuracy: 0.2920
7.962458848953247
[TRAIN]  Epoch [3/400]   Loss: 1.9454     Accuracy: 0.3425
[Test]  Epoch [3/400]   Accuracy: 0.3120
7.862977981567383
[TRAIN]  Epoch [4/400]   Loss: 1.5665     Accuracy: 0.4251
[Test]  Epoch [4/400]   Accuracy: 0.4051
7.885333299636841
[TRAIN]  Epoch [5/400]   Loss: 1.3249     Accuracy: 0.4699
[Test]  Epoch [5/400]   Accuracy: 0.5128
7.427367925643921
[TRAIN]  Epoch [6/400]   Loss: 1.2250     Accuracy: 0.5187
[Test]  Epoch [6/400]   Accuracy: 0.5894
5.840807676315308
[TRAIN]  Epoch [7/400]   Loss: 1.1303     Accuracy: 0.5758
[Test]  Epoch [7/400]   Accuracy: 0.6168
5.839746713638306
[TRAIN]  Epoch [8/400]   Loss: 1.0714     Accuracy: 0.6073
[Test]  Epoch [8/400]   Accuracy: 0.5949
5.816826105117798
[TRAIN]  Epoch [

In [ ]:
plt.figure()
plt.plot(range(1, trainer.epochs + 1), trainer.train_losses, label='Training losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
    

In [ ]:
trainer.train_losses

In [ ]:
print(adj.shape)

In [ ]:
print('Test Accuracies')
print(trainer.testaccuracies)
print('Train Losses')
print(trainer.train_losses)
print('Train Accuracies')
print(trainer.trainaccuracies)